# Install dependencies

In [ ]:
!pip install fairseq
!pip install sentencepiece
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 49.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.2/286.2 kB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.4/112.4 kB 12.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fairseq: filename=fairseq-0.12.2-cp310-cp310-linux_x86_64.whl size=11291803 sha256=53f0b0e970f12d8c71cb7d946ecc390f8bf5b16608d10794679d277b7f9f8f56
  Stored in directory: /root/.cache/pip/wheels/e4/35/55/9c66f65ec7c83fd6fbc2b9502a0ac81b2448a1196159dacc32
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.8-py3-none-any.whl size=141209 sha256=138d68344860476714785d10a6483a4f7878ad550815bcab47d1323c7b666f34
  S

In [ ]:
!git clone https://github.com/google/sentencepiece.git
!mkdir sentencepiece/build
!cd sentencepiece/build && cmake ..
!cd sentencepiece/build && make -j $(nproc)

Cloning into 'sentencepiece'...
remote: Enumerating objects: 4823, done.
remote: Counting objects: 100% (1450/1450), done.
remote: Compressing objects: 100% (320/320), done.
remote: Total 4823 (delta 1175), reused 1195 (delta 1089), pack-reused 3373
Receiving objects: 100% (4823/4823), 26.77 MiB | 6.33 MiB/s, done.
Resolving deltas: 100% (3314/3314), done.
CMake Deprecation Warning at CMakeLists.txt:15 (cmake_minimum_required):
  Compatibility with CMake < 3.5 will be removed from a future version of
  CMake.

  Update the VERSION argument <min> value or use a ...<max> suffix to tell
  CMake that the project does not need compatibility with older versions.


-- VERSION: 0.2.00
-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX co

In [ ]:
%env SPM=/content/sentencepiece/build/src
!echo $SPM

env: SPM=/content/sentencepiece/build/src
/content/sentencepiece/build/src


In [ ]:
!wget https://dl.fbaipublicfiles.com/fairseq/models/mbart/mbart.cc25.v2.tar.gz
!tar -xzvf mbart.cc25.v2.tar.gz

--2023-08-27 02:49:30--  https://dl.fbaipublicfiles.com/fairseq/models/mbart/mbart.cc25.v2.tar.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.227.219.70, 13.227.219.33, 13.227.219.10, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|13.227.219.70|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5618826847 (5.2G) [application/gzip]
Saving to: ‘mbart.cc25.v2.tar.gz’

mbart.cc25.v2.tar.g 100%[===================>]   5.23G   299MB/s    in 18s     

2023-08-27 02:49:47 (306 MB/s) - ‘mbart.cc25.v2.tar.gz’ saved [5618826847/5618826847]

mbart.cc25.v2/
mbart.cc25.v2/sentence.bpe.model
mbart.cc25.v2/dict.txt
mbart.cc25.v2/model.pt


In [ ]:
# Import neccessary libs
import os
import torch
from glob import glob
from typing import List

from google.colab import drive
from fairseq.data import Dictionary
from fairseq.tokenizer import tokenize_line

In [ ]:
# Mount from drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Create stored data folder
!mkdir tokenized
!mkdir model

# Dataset pre-processing
1. Split dataset to train, val, test set
1.   Using the pretrained Sentencepiece model to pre-process our data (also do this to validation/test sets). It will split the sentences into **subwords**, adding the special symbol `▁` to the first subword of a word.





In [ ]:
!$SPM/spm_encode --model="drive/MyDrive/sentencepiece.bpe.model" --output_format=piece < "drive/MyDrive/en-vi/train.en" > tokenized/train.spm.en
!$SPM/spm_encode --model="drive/MyDrive/sentencepiece.bpe.model" --output_format=piece < "drive/MyDrive/en-vi/train.vi" > tokenized/train.spm.vi

!$SPM/spm_encode --model="drive/MyDrive/sentencepiece.bpe.model" --output_format=piece < "drive/MyDrive/en-vi/valid.en" > tokenized/val.spm.en
!$SPM/spm_encode --model="drive/MyDrive/sentencepiece.bpe.model" --output_format=piece < "drive/MyDrive/en-vi/valid.vi" > tokenized/val.spm.vi

# Pruning the pre-trained model
Most of the words in the large vocabulary used by the original pre-training model are not actually used in the finetune process, so this part of redundant information can be removed.

Reduce the size of the pre-trained model by pruning the word embeddings for fine-tuning:

- Firstly, build a new vocab for our dataset

In [ ]:
# Build new vocab
def pad_dict(d: Dictionary, num_extra_symbols: int, padding_factor: int = 8) -> None:
    i = 0
    while (len(d) + num_extra_symbols) % padding_factor != 0:
        symbol = f"madeupword{i:04d}"
        d.add_symbol(symbol, n=0)
        i += 1

langs = 'ar_AR,cs_CZ,de_DE,en_XX,es_XX,et_EE,fi_FI,fr_XX,gu_IN,hi_IN,it_IT,ja_XX,kk_KZ,ko_KR,lt_LT,lv_LV,my_MM,ne_NP,nl_XX,ro_RO,ru_RU,si_LK,tr_TR,vi_VN,zh_CN'
data_dir = "./tokenized/*.spm.*"
output = './model/dict.txt'

langs = langs.split(",")
ft_dict = Dictionary()

for data_path in glob(data_dir):
  Dictionary.add_file_to_dictionary(data_path, ft_dict, tokenize_line, 4)

ft_dict.finalize(padding_factor=0)
pad_dict(ft_dict, len(langs) + 1)
ft_dict.save(output)

- Pruning word embeddings base on new vocab built

In [ ]:
# Pruning word embeddings by the new vocab

def load_dict(langs: List[str], path: str) -> Dictionary:
    d = Dictionary.load(path)
    for l in langs:
        d.add_symbol(f"[{l}]")
    d.add_symbol("<mask>")
    return d

pre_train_dir = './mbart.cc25.v2'
ft_dict_path = './model/dict.txt'
output = './model/model.pt'

pre_dict = load_dict(langs, os.path.join(pre_train_dir, "dict.txt"))
ft_dict = load_dict(langs, ft_dict_path)
data = torch.load(os.path.join(pre_train_dir, "model.pt"))
model = data["model"]
mapping: List[int] = []

for i in range(len(ft_dict)):
    word = ft_dict[i]
    mapping.append(pre_dict.index(word))

for name in ["encoder.embed_tokens.weight", "decoder.embed_tokens.weight"]:
    pre_tensor: torch.Tensor = model[name]
    ft_tensor = torch.zeros(
        [len(ft_dict), 1024], dtype=pre_tensor.dtype, layout=pre_tensor.layout, device=pre_tensor.device,
    )
    for ft_i, pre_i in enumerate(mapping):
      ft_tensor[ft_i] = pre_tensor[pre_i]
    model[name] = ft_tensor

torch.save(data, output)

# Binarize pre-process data

Binarize the data to the Fairseq format.

In [ ]:
!fairseq-preprocess \
  --source-lang "en" \
  --target-lang "vi" \
  --trainpref "tokenized/train.spm" \
  --validpref "tokenized/val.spm" \
  --destdir "bin" \
  --bpe sentencepiece \
  --thresholdtgt 0 \
  --thresholdsrc 0 \
  --srcdict "model/dict.txt" \
  --tgtdict "model/dict.txt" \
  --workers 70

2023-08-27 15:51:43.708281: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-08-27 15:51:45 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2023-08-27 15:51:45 | INFO | fairseq_cli.preprocess | Namespace(no_progress_bar=False, log_interval=100, log_format=None, log_file=None, aim_repo=None, aim_run_hash=None, tensorboard_logdir=None, wandb_project=None, azureml_logging=False, seed=1, cpu=False, tpu=False, bf16=False, memory_efficient_bf16=False, fp16=False, memory_efficient_fp16=False, fp16_no_flatten_grads=False, fp16_init_scale=128, fp16_scale_window=None, fp16_scale_tolerance=0.0, on_cpu_convert_precision=False, min_loss_scale=0.0001, threshold_loss_scale=None, amp=False, amp_batch_retries=2, amp_init_scale=128, amp_scale_window=None, user_dir=None, empty_cache_freq=0, all_gather_list_size=16384, model_parallel_size=1, quantization_config_path=None, profile=False, reset_logging=False, s

# Training

Enable half precision (`--fp16`) or define/use a smaller model to speed up the training. But in case using (`--memory-efficient-fp16`) for using memory more efficient

In the case of Colab timing out, change the `--keep-interval-updates` and `--no-epoch-checkpoints` flags to save intermidate checkpoints and then resume the training from the last checkpoint.

In [ ]:
# %reset -f # Release memory

In [ ]:
!fairseq-train \
  "bin" \
  --encoder-normalize-before --decoder-normalize-before \
  --arch mbart_large --layernorm-embedding \
  --task translation_from_pretrained_bart \
  --source-lang en --target-lang vi \
  --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
  --dataset-impl mmap \
  --optimizer adam --adam-eps 1e-06 --adam-betas '(0.9, 0.98)' \
  --lr-scheduler polynomial_decay --lr 5e-05 --warmup-updates 2500 --total-num-update 40000 \
  --dropout 0.3 --attention-dropout 0.1 --weight-decay 0.0 \
  --max-tokens 4000 --update-freq 2 \
  --save-interval 1 --save-interval-updates 3000 --keep-interval-updates 10 --no-epoch-checkpoints \
  --seed 222 --log-format simple --log-interval 2 \
  --restore-file drive/MyDrive/en-vi/checkpoint_2_9000.pt \
  --reset-optimizer --reset-meters --reset-dataloader --reset-lr-scheduler \
  --langs ar_AR,cs_CZ,de_DE,en_XX,es_XX,et_EE,fi_FI,fr_XX,gu_IN,hi_IN,it_IT,ja_XX,kk_KZ,ko_KR,lt_LT,lv_LV,my_MM,ne_NP,nl_XX,ro_RO,ru_RU,si_LK,tr_TR,vi_VN,zh_CN \
  --memory-efficient-fp16 --save-dir drive/MyDrive/en-vi

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
2023-08-27 07:39:28 | INFO | train_inner | epoch 002:   4970 / 5536 loss=2.513, nll_loss=0.635, ppl=1.55, wps=5418.5, ups=0.72, wpb=7567.5, bsz=316, num_updates=10486, lr=3.9352e-05, gnorm=0.541, loss_scale=0.5, train_wall=3, gb_free=3.9, wall=16860
2023-08-27 07:39:31 | INFO | train_inner | epoch 002:   4972 / 5536 loss=2.549, nll_loss=0.667, ppl=1.59, wps=5377.9, ups=0.69, wpb=7814, bsz=260, num_updates=10488, lr=3.93493e-05, gnorm=0.568, loss_scale=0.5, train_wall=3, gb_free=3.8, wall=16863
2023-08-27 07:39:34 | INFO | train_inner | epoch 002:   4974 / 5536 loss=2.531, nll_loss=0.641, ppl=1.56, wps=4790.3, ups=0.73, wpb=6526.5, bsz=112, num_updates=10490, lr=3.93467e-05, gnorm=0.707, loss_scale=0.5, train_wall=3, gb_free=3.3, wall=16865
2023-08-27 07:39:37 | INFO | train_inner | epoch 002:   4976 / 5536 loss=2.547, nll_loss=0.668, ppl=1.59, wps=5350.1, ups=0.7, wpb=7592, bsz=264, num_updates=10492, lr=3.9344e-05, gnorm=0.565, 